# Profiling and discussion

## Lorenz 96

In [81]:
# Add profiling code here
import numpy as np

def lorenz96(initial_state, nsteps, constants=(1/101, 100, 8)):
    """
    Perform iterations of the Lorenz 96 update.

    Parameters
    ----------
    initial_state : array_like or list
        Initial state of lattice in an array of floats.
    nsteps : int
        Number of steps of Lorenz 96 to perform.

    Returns
    -------
    numpy.ndarray
        Final state of lattice in an array of floats
    """

    alpha, beta, gamma = constants
    state = np.array(initial_state, dtype=float)
    N = len(state)
    new_state = np.empty_like(state)  # Create a new state array

    for _ in range(nsteps):
        # Compute the first two elements
        new_state[0] = alpha * (beta * state[0] + (state[N - 2] - state[1]) * state[N - 1] + gamma)
        new_state[1] = alpha * (beta * state[1] + (state[N - 1] - state[0]) * state[0] + gamma)

        # Compute the elements between 2 and N-2
        new_state[2:N - 2] = alpha * (beta * state[2:N - 2] + (state[0:N - 4] - state[3:N - 1]) * state[1:N - 3] + gamma)   

        # Compute the last element
        new_state[N - 1] = alpha * (beta * state[N - 1] + (state[N - 3] - state[0]) * state[N - 2] + gamma)

        # Update the state array
        state[:] = new_state

    return state


initial_state = np.full(49, 8.0)
initial_state = np.insert(initial_state,2,9.0)
nsteps = 50

In [82]:
%timeit lorenz96(initial_state, nsteps)

/var/folders/g1/_5ywfh994dvfccvkj4cn_wy00000gp/T/ipykernel_24384/258648170.py:32: RuntimeWarning: overflow encountered in multiply
  new_state[2:N - 2] = alpha * (beta * state[2:N - 2] + (state[0:N - 4] - state[3:N - 1]) * state[1:N - 3] + gamma)
/var/folders/g1/_5ywfh994dvfccvkj4cn_wy00000gp/T/ipykernel_24384/258648170.py:32: RuntimeWarning: invalid value encountered in subtract
  new_state[2:N - 2] = alpha * (beta * state[2:N - 2] + (state[0:N - 4] - state[3:N - 1]) * state[1:N - 3] + gamma)
/var/folders/g1/_5ywfh994dvfccvkj4cn_wy00000gp/T/ipykernel_24384/258648170.py:32: RuntimeWarning: invalid value encountered in add
  new_state[2:N - 2] = alpha * (beta * state[2:N - 2] + (state[0:N - 4] - state[3:N - 1]) * state[1:N - 3] + gamma)


241 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Add discussion here

### First Version
By interacting with chatgpt I realised that explcitly stating the data type allows numpy to optimise operations pertaining to the specific data. After this i used the timeit to profile my code and got a speed of 166 microseconds.
### Second Version
Intead of using nested loops i decided to use numpys abilties to "vectorize" the function in order to optimise.After vecotrising i found that the code ran slower for smaller steps(20) but faster than my first version for larger steps(50)
### Third Version
After consulting chatgpt it seems the action of change a new array is inneficient due to memory allocation and deallocation. This didnt seem to make the program as efficient.
### Fourth Version
Now that ive applied numpys capabilities where i can to optimise. I decided to try an optimise the actual algorithm. Instead of using one algorithm for every part of the array the calculation was split to the first, second, last and all the arrays in between. This allowed a much faster vecotrisation process.

Compared to my original code I have managed to optimise my code to go from 1.03 milliseconds to 228 microseconds for the same array.. I have changed the algorithm and used vectorisation where i could to make it more efficient.


#### Testing
When creating the tests for this function I used the assignment pdf to give me my basic test cases. I then used a different set of constants to calculate by hand and assert the correct output array

## Game of Life

In [65]:
import numpy as np

#version 2
def life(initial_state, nsteps, rules="basic", periodic=False):
    """
    Perform iterations of Conway’s Game of Life.
    Parameters
    ----------
    initial_state : array_like or list of lists
        Initial 2d state of grid in an array of ints.
    nsteps : int
        Number of steps of Life to perform.
    rules : str
        Choose a set of rules from "basic", "2colour" or "3d".
    periodic : bool
        If True, use periodic boundary conditions.
    Returns
    -------
    numpy.ndarray
         Final state of grid in array of ints.
    """

    # write your code here to replace return statement
    state = np.array(initial_state, dtype=int)

    # Determine if we're working in 2D or 3D
    if rules == "3d":
        if state.ndim != 3:
            raise ValueError("Invalid grid dimension!")
        rows, cols, depth = state.shape
        for _ in range(nsteps):
            next_state = state.copy()
            
            for i in range(rows):
                for j in range(cols):
                    for k in range(depth):
                        total = 0  # Count the neighbors

                        for x in [-1, 0, 1]:
                            for y in [-1, 0, 1]:
                                for z in [-1, 0, 1]:
                                    if x == 0 and y == 0 and z == 0:
                                        continue  # Skip the current cell
                                    ni, nj, nk = i + x, j + y, k + z
                                    if periodic:  # Handle periodic boundary conditions
                                        ni %= rows
                                        nj %= cols
                                        nk %= depth
                                    elif ni < 0 or ni >= rows or nj < 0 or nj >= cols or nk < 0 or nk >= depth:
                                        continue  # Skip out of bounds

                                    total += state[ni, nj, nk]
                                    
                            if state[i, j, k] != 0:  # If cell is alive
                                if total < 5 or total > 6:  # Die
                                    next_state[i, j, k] = 0
                            else:  # Dead cell
                                if total == 4:
                                    next_state[i, j, k] = 1  # Birth
            
            state = next_state
        return state
    else:
        if state.ndim != 2:
            raise ValueError("Invalid grid dimension!")
        rows, cols = state.shape
        for _ in range(nsteps):
            next_state = state.copy()

            for i in range(rows):
                for j in range(cols):
                    total = 0
                    blue_neighbours = 0
                    red_neighbours = 0
                    for x in [-1, 0, 1]:
                        for y in [-1, 0, 1]:
                            if x == 0 and y == 0:
                                continue
                            ni, nj = i + x, j + y
                            if periodic:
                                ni %= rows
                                nj %= cols
                            elif ni < 0 or ni >= rows or nj < 0 or nj >= cols:
                                continue

                            if state[ni][nj] > 0:
                                total += 1
                                if state[ni][nj] == 1:
                                    blue_neighbours += 1
                                else:
                                    red_neighbours += 1

                    if rules == "basic":
                        if state[i][j] == 1:  # If cell is alive
                            if total < 2 or total > 3:  # Die
                                next_state[i][j] = 0
                        else:  # Dead cell
                            if total == 3:
                                next_state[i][j] = 1

                    elif rules == "2colour":
                        if state[i][j] == 1 or state[i][j] == 2:  # If cell is alive
                            if total < 2 or total > 3:  # Die
                                next_state[i][j] = 0
                        else:  # Dead cell
                            if total == 3:
                                if blue_neighbours > red_neighbours:
                                    next_state[i][j] = 1
                                else:
                                    next_state[i][j] = 2

            state = next_state

        return state

In [66]:
%timeit life(X, 10,rules = "basic",periodic=False)  # Modify as needed

398 µs ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Add discussion here
The first iteration of the code i recieved from chatgpt was riddled with bugs and missing logic such as code for seperate colour in the "2d colour" extension. I first created a skeleton of code that ensured that any array would inputter correctly depending on which extension was specified. The array is unpacked into rows, columns and potentially height/depth. once this skeleton was created i focused 2D version of the code first. One thing to note is that allthough chatgpt gave me seperate function it was decided in order not call functions within a function which can cause overhead i combined all code under one fiunction with conditonals to split the two. This reduced the readbility of the code and is not ideal overall but should allow the timeit function to display a lower number. There were other standard code changes made in order to reduce redundacy and improve the code and the original code was not ideal at all.

Once the code was working and running which was ensured using the testing file and test cases several steps were taken to attempt to optimise the way in which the code worked. Firstly conditionals and logics path were condensed to minimise complexity for computations. Modular arithmetic was used to handle periodic boundary conditions to reduce the need for more conditonals. Numpy arrays and operations were used where they could be which is computationally faster than the standard python loops.

One significant issue that was not understood was the indexing. When I used a faster array indexing the runtime seemed to take longer not shorter as expected. The test cases still worked and gpt was not able to aid with this issue.